# ***Описание работы:***
В данной работе, в учебных целях, исследуется набор данных "Отток клиентов банка" для анализа причин оттока клиентов и получения информации, которая поможет организации улучшить стратегии удержания клиентов. Понимание факторов, способствующих оттоку, имеет решающее значение для сохранения клиентов и поддержания прибыльности в долгосрочной перспективе.

***Основные этапы работы:***
* Загрузка и изучение структуры данных и их предварительная обработка.
* Исследовательский анализ данных (EDA): углубленный анализ набора данных, включая визуализации и статистические сводки, необходимые для получения представления о распределении и взаимосвязях между переменными.
* Интерпретация результатов исследования и подведение итогов: извлечение полезной информации из исследования и составление кратких, содержательных выводов.

### *Описание набора данных*

* RowNumber — Соответствует номеру (индексу) строки и не влияет на результат исследований.
* CustomerId — Содержит уникальный идентификатор клиента банка и не влияет на результат исследований.
* Surname — Полное имя клиента.
* CreditScore — Кредитный рейтинг клиента.
* Geography — Местоположение клиента.
* Gender — Пол клиента.
* Age — Возраст клиента.
* Tenure — Количество лет, в течение которых человек является клиентом банка.
* Balance — Остаток (текущий баланс) средств на счете клиента. Валюта - предположительно, Eвро (знак валюты - €).
* NumOfProducts — Количество продуктов, которые приобрел клиент.
* HasCrCard — Указывает, есть ли у клиента банка кредитная карта.
* IsActiveMember — Критерий, указывающий, является ли человек "активным клиентом".
* EstimatedSalary — Уровень заработной платы клиента (Eвро/год).
* Exited — Критерий, указывающий, является ли человек, в настоящее время, клиентом банка.
* Complain — Указывает, есть ли жалоба у клиента.
* Satisfaction Score — Оценка качества обслуживания, предоставленная клиентом.
* Card Type — Тип дебетовой карты клиента.
* Points Earned — Накопительный баллы (бонусы), заработанные клиентом.


# *Загрузка и изучение структуры данных и их предварительная обработка*

In [ ]:
import numpy as np                 # Импорт библиотеки NumPy
import pandas as pd                # Импорт библиотеки Pandas
import seaborn as sns              # Импорт библиотеки SeaBorn
import matplotlib.pyplot as plt    # Импорт библиотеки MatPlotLib
from scipy import stats            # Импорт модуля stats из библиотеки Scipy

plt.style.use('bmh')               # Устанавлием стиль графиков (Bayesian Methods for Hackers) по умолчанию

In [ ]:
df = pd.read_csv('/kaggle/input/bank-customer-churn/Customer-Churn-Records.csv')   # Создание датафрейма
df.head(5)                                                                         # Отображение нескольких строк датафрейма

In [ ]:
df.info()    # Просмотр структуры датафрейма

In [ ]:
df.isna().mean()   # Убеждаемся в отсутствии пустых значений в строках

In [ ]:
df = df.drop(['RowNumber','CustomerId','Surname'], axis = 1)     # Удаление столбцов, не влияющих на исследование

df_only_numeric = df.drop(['Geography','Card Type'],axis = 1)    # Создание датафрейма только с числовыми типами данных

df_only_numeric.loc[df['Gender'] == 'Male', 'Gender'] = 1        # Изменение значения 'Male' на значения '1'
df_only_numeric.loc[df['Gender'] == 'Female', 'Gender'] = 0      # Изменение значения 'Female' на значения '0'

df_only_numeric = df_only_numeric.astype({'Gender':int})         # Преобразование строкового типа данных в числовой

df_boolean = df.astype({'HasCrCard': bool, 'IsActiveMember': bool, 'Exited': bool, 'Complain':bool})    # Преобразование числового типа данных в логический

Датафрейм - df_only_numeric удобен для применения некоторых методов, таких как - describe() и corr() и т.п., которые будут использованы в дальнейшем исследовании.
В датафрейме - df_boolean изменяем числовые значения 1 и 0 на логические True и False соответственно.  

# *Исследовательский анализ данных*

In [ ]:
df_only_numeric.describe()    # Просмотр статистистических данных

In [ ]:
df_matrix = df_only_numeric.corr()    # Присваивание имени корреляционной матрице
display(df_matrix)                    # Отображение матрицы в виде таблицы
sns.heatmap(df_matrix, annot = True, fmt='.1f',vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', linewidths=1, linecolor='black',xticklabels=df_matrix.columns)
plt.xticks(rotation=45, ha="right")
plt.show()                            # Визуализация матрицы корреляции

Данная матрица показывает существование линейных связей между переменными. Как видно, наиболее сильная связь (линейная зависимость) между столбцами 'Exited' и 'Complain'.

In [ ]:
plt.figure(figsize=(5, 5))                                                                                                  # Задаем размеры диаграммы

plt.pie(df['Exited'].value_counts(), labels=['Действующие клиенты', 'Ушедшие клиенты'], autopct='%1.2f%%', startangle=90)   # Создание круговой диаграммы
plt.title('Распределение оттока клиентов')                                                                                  # Назначаем название диаграммы
plt.show() 

display(df_boolean['Exited'].value_counts())                                                                                # Подсчет действующих и ушедших клиетов банка и отображение в виде таблицы

В итоге имеем - 20,38% ушедших клиентов (2038 из 10000) и 79,62% действующих (7962 из 10000). Начнем исследование зависимостей оттока клиентов по порядку, начиная с первого стоблца датафрейма (CreditScore).

In [ ]:
df_boolean['Age_range'] = df['Age_range'] = pd.cut(df['Age'],bins=[17,25,35,45,60,92])    # Создание столбцов в датафреймах с диапазоном возраста клиентов (Границы диапазов установлены согласно мин и макс значениям).

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))                               # Задаем расположение и размеры графиков на графической сетке

df_boolean.query("Exited == True")['CreditScore'].hist(ax=axes[0], bins = 25)             # Создание гистограммы ушедших клиентов и их кредитного рейтинга
axes[0].set_xlabel('Кредитный рейтинг')                                                   # Задаем название оси x
axes[0].set_ylabel('Количество ушедних клиентов')                                         # Задаем название оси y

df_boolean.query("Exited == False")['CreditScore'].hist(ax=axes[1], bins = 25)            # Создание гистограммы действующих клиентов и их кредитного рейтинга
axes[1].set_xlabel('Кредитный рейтинг')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

display(df_boolean.query("Exited == True").groupby('Age_range')['CreditScore'].agg(['count','mean','median']).reset_index())     # Подсчет ушедших клиентов, среднего значение и медианы их кредитного рейтинга
display(df_boolean.query("Exited == False").groupby('Age_range')['CreditScore'].agg(['count','mean','median']).reset_index())    # Подсчет действующих клиентов, среднего значение и медианы их кредитного рейтинга

Как видно, различия кредитного рейтинга между ушедшими и действующими клиентами практически отсутствуют. Следовательно, кредитный рейтинг не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
sns.countplot(x='Geography', hue='Exited', data=df_boolean)                                  # Создание столбчатого графика по странам
plt.xlabel('Страна')
plt.ylabel('Количество клиентов')
plt.show()
 
df_country = df.groupby('Geography')['Exited'].agg(['count','mean']).reset_index()           # Создание датафрейма с группировкой по странам
df_country = df_country.assign(exited_clients = df_country['count'] * df_country['mean'])    # Добавление столбца с подсчетом ушедших клиентов для каждой страны
display(df_country.sort_values(by = 'exited_clients', ascending = False))                    # Сортировка столбца 'exited_clients' по убыванию

Согласно этому набору данных, можно сделать следующие выводы:
* Наибольший отток клиентов наблюдается в Германии - 32,44% (814 из 2509 клиентов)
* В оставшихся странах доля оттока клиентов приблизительно равна - 16,67% для Испании (413 из 2477 клиентов) и 16,17% для Франции (811 из 5014 клиентов)

In [ ]:
sns.countplot(x='Gender', hue='Exited', data=df_boolean)                                 # Создание столбчатого графика по полу
plt.xlabel('Пол клиента')
plt.ylabel('Количество клиентов')
plt.show()

df_gender = df.groupby('Gender')['Exited'].agg(['count','mean']).reset_index()           # Создание датафрейма с группировкой по полу
df_gender = df_gender.assign(exited_clients = df_gender['count'] * df_gender['mean'])    # Добавление столбца с подсчетом ушедших клиентов для каждого пола
display(df_gender)

Согласно этому набору данных, можно сделать следующие выводы:
* Наибольший отток наблюдается у клиентов-женщин  - 25,07% (1139 из 4543 клиентов)
* У мужчин доля ушедших клиентов равна - 16,47% (899 из 5457 клиентов)

In [ ]:
sns.countplot(x='Age_range', hue='Exited', data=df_boolean)                                                                # Создание столбчатого графика по диапазону возрастов
plt.xlabel('Возраст')
plt.ylabel('Количество клиентов')
plt.show()

df_Age_range = df_boolean.groupby('Age_range')['Exited'].agg(['count','mean']).reset_index()                               # Создание датафрейма с группировкой по диапазону возрастов
df_Age_range_male = df.query('Gender == "Male"').groupby('Age_range')['Exited'].agg(['count','mean']).reset_index()        # Создание датафрейма с группировкой по диапазону возрастов для клиентов - мужчин
df_Age_range_female = df.query('Gender == "Female"').groupby('Age_range')['Exited'].agg(['count','mean']).reset_index()    # Создание датафрейма с группировкой по диапазону возрастов для клиентов - женщин

df_Age_range = df_Age_range.assign(exited_clients = df_Age_range['count'] * df_Age_range['mean'])                          # Добавление столбца с подсчетом ушедших клиентов
df_Age_range = df_Age_range.assign(exited_clients_male = df_Age_range_male['count'] * df_Age_range_male['mean'])           # Добавление столбца с подсчетом ушедших клиентов - мужчин
df_Age_range = df_Age_range.assign(exited_clients_female = df_Age_range_female['count'] * df_Age_range_female['mean'])     # Добавление столбца с подсчетом ушедших клиентов - женщин
display(df_Age_range)

Согласно этому набору данных, можно сделать следующие выводы:
* Наибольший отток наблюдается у клиентов в возрасте от 46 до 60 лет  - 51,12% (842 из 1647 клиентов)
* Заметен рост оттока клиентов при переходе к все более возрастным группам (до 60 лет). Далее доля оттока уменьшается.
* Учитывая долю оттока и количество клиентов для каждой возрастной группы, можно утверждать, что наиболее предпочтительным клиентов для банка является мужчина в возрасте от 26 до 35 лет включительно.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

df_boolean.query("Exited == True")['Tenure'].hist(ax=axes[0])    # Создание гистограммы ушедших клиентов и их срока пребывания в банке
axes[0].set_xlabel('Срок пребывания клиента в банке')
axes[0].set_ylabel('Количество ушедних клиентов')

df_boolean.query("Exited == False")['Tenure'].hist(ax=axes[1])   # Создание гистограммы действующих клиентов и их срока пребывания в банке
axes[1].set_xlabel('Срок пребывания клиента в банке')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

display(df_boolean.query("Exited == True").groupby('Age_range')['Tenure'].agg(['count','mean']).reset_index())     # Подсчет ушедших клиентов и среднего значение их срока пребывания в банке
display(df_boolean.query("Exited == False").groupby('Age_range')['Tenure'].agg(['count','mean']).reset_index())    # Подсчет действующих клиентов и среднего значение их срока пребывания в банке

Как видно, различия срока пребывание в качестве клиента банка между ушедшими и действующими клиентами практически отсутствуют. Следовательно, срок пребывания в банке не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

df_boolean.query("Exited == True")['Balance'].hist(ax=axes[0],bins = 25)    # Создание гистограммы ушедших клиентов и их баланса
axes[0].set_xlabel('Баланс')            
axes[0].set_ylabel('Количество ушедних клиентов')                

df_boolean.query("Exited == False")['Balance'].hist(ax=axes[1], bins = 25)   # Создание гистограммы действующих клиентов и их текущего баланса
axes[1].set_xlabel('Текущий баланс')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

Теперь посмотрим на аналогичное распределение у клиентов с балансом строго больше нуля.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

df_boolean.query("Exited == True and Balance > 0")['Balance'].hist(ax=axes[0],bins = 50)
axes[0].set_xlabel('Баланс')            
axes[0].set_ylabel('Количество ушедних клиентов')                

df_boolean.query("Exited == False and Balance > 0")['Balance'].hist(ax=axes[1], bins = 50)
axes[1].set_xlabel('Текущий баланс')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

sns.boxplot(x='Exited', y='Balance', data=df_boolean,ax=axes[0])                         # Построение диаграммы распределения баланса у клиентов        
axes[0].set_ylabel('Баланс')

sns.boxplot(x='Exited', y='Balance', data=df_boolean.query('Balance > 0'),ax=axes[1])    # Построение диаграммы распределения баланса у клиентов, где баланс строго больше нуля       
axes[1].set_ylabel('Баланс')

plt.show()

In [ ]:
display(df_only_numeric.query('Balance == 0')['Exited'].agg(['count','mean']).reset_index())    # Подсчет клиентов с нулевым балансом и доли ушедших клиентов
display(df_only_numeric.query('Balance > 0')['Exited'].agg(['count','mean']).reset_index())     # Подсчет клиентов с балансом строго больше нуля и доли ушедших клиентов

Согласно этому набору данных, можно сделать следующие выводы:
* Распределение баланса не показывает четкого различия между ушедшими и действующими клиентами
* Доля ушедших клиентов с нулевым балансом в 1,74 раза меньше (24,1% / 13,8), чем доля ушедших клиентов с балансом строго больше нуля

In [ ]:
df.query('Balance > 0')['Balance'].hist(bins = 50)    # Создание гистограммы для клиентов, у которых баланс строго больше нуля
plt.xlabel('Баланс')
plt.ylabel('Количество клиентов')

На данной гистограмме распределение значений стремится к нормальному. Убедимся в этом наверняка.

In [ ]:
df_balance_normal_dist = df.query('Balance > 0')['Balance']

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

mean = df_balance_normal_dist.mean()                                          # Находим среднее значения баланса
std = df_balance_normal_dist.std()                                            # Находим среднеквадратичное отклонение баланса

sns.kdeplot(df_balance_normal_dist,ax=axes[0])                                # Создание графика плотности распределения (PDF)
axes[0].set_xlabel('Баланс')            
axes[0].set_ylabel('Плотность')
axes[0].axvline(mean, color='g', linestyle='--', label='Mean')                # Определение параметров линии среднего значения на графике
axes[0].axvline(mean + std, color='b', linestyle='--', label='Mean + Std')    # Определение параметров линии среднеквадратичного отклонение на графике
axes[0].axvline(mean - std, color='b', linestyle='--', label='Mean - Std')    
axes[0].legend()                                                              # Отображение легенды на графике

sns.ecdfplot(df_balance_normal_dist,ax=axes[1])                               # Создание графика функции распределения (CDF)
axes[1].set_xlabel('Баланс')            
axes[1].set_ylabel('Доля')
axes[1].axvline(mean, color='g', linestyle='--', label='Mean')
axes[1].axvline(mean + std, color='b', linestyle='--', label='Mean + Std')
axes[1].axvline(mean - std, color='b', linestyle='--', label='Mean - Std')
axes[1].legend()

plt.show()

In [ ]:
stats.probplot(df_balance_normal_dist,dist='norm',plot=plt)    # Строим график совпадения выборочных значений по отношению квантилям заданного теоретического распеделения
plt.xlabel('Квантиль')
plt.ylabel('Баланс')
plt.title('График совпадений')
plt.show()

В итоге можно утверждать, что баланс клиентов со значениями строго больше нуля имеет нормальное распределение.

In [ ]:
sns.countplot(x='NumOfProducts', hue='Exited', data=df_boolean)                                    # Создание столбчатого графика по количеству купленных продуктов     
plt.xlabel('Количество купленных продуктов')
plt.ylabel('Количество клиентов')
plt.show()

df_products = df_boolean.groupby('NumOfProducts')['Exited'].agg(['count','mean']).reset_index()    # Создание датафрейма с группировкой по количеству купленных продуктов
df_products = df_products.assign(exited_clients = df_products['count'] * df_products['mean'])      # Добавление столбца с подсчетом ушедших клиентов
display(df_products)

Согласно этому набору данных, можно сделать следующие выводы:
* Большая часть клиентов купившая более 2 продуктов ушла из банка (82,7% для 3 продуктов и 100% для 4 продуктов), но стоит заметить, что суммарное количество ушедших составляет всего 280 из 2038 (13,7%) клиентов покинувших банк.
* Наибольшое количество ушедших клиентов имели всего один купленный продукт - 27,7% (1409 из 5084 клиентов)

In [ ]:
sns.countplot(x='HasCrCard', hue='Exited', data=df_boolean)                                # Создание столбчатого графика по наличию кредитной карты
plt.xlabel('Наличие кредитной карты')
plt.ylabel('Количество клиентов')
plt.show()

df_card = df_boolean.groupby('HasCrCard')['Exited'].agg(['count','mean']).reset_index()    # Создание датафрейма с группировкой по наличию кредитной карты
df_card = df_card.assign(exited_clients = df_card['count'] * df_card['mean'])              # Добавление столбца с подсчетом ушедших клиентов
display(df_card.sort_values(by = 'exited_clients', ascending = False))

Как видно, доля ушедших клиентов между клиентами с кредитной картой и без нее практически равна (~20%). Следовательно, наличие кредитной карты не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
sns.countplot(x='IsActiveMember', hue='Exited', data=df_boolean)                                  # Создание столбчатого графика по активности клиента
plt.xlabel('Активность клиента')
plt.ylabel('Количество клиентов')
plt.show()

df_active = df_boolean.groupby('IsActiveMember')['Exited'].agg(['count','mean']).reset_index()    # Создание датафрейма с группировкой по активности клиента
df_active = df_active.assign(exited_clients = df_active['count'] * df_active['mean'])             # Добавление столбца с подсчетом ушедших клиентов
display(df_active.sort_values(by = 'exited_clients', ascending = False))

Согласно этому набору данных, можно сделать следующие выводы:
* Менее активные клиенты чаще уходят из банка - 26,87% (1303 из 4849 клиентов)
* Доля ушедших активных клиентов заметно меньше - 14,27% (735 из 5151 клиента)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

df_boolean.query("Exited == True")['EstimatedSalary'].hist(ax=axes[0],bins = 25)     # Создание гистограммы ушедших клиентов и их заработной платы
axes[0].set_xlabel('Доход')            
axes[0].set_ylabel('Количество ушедних клиентов')                

df_boolean.query("Exited == False")['EstimatedSalary'].hist(ax=axes[1], bins = 25)   # Создание гистограммы действующих клиентов и их заработной платы
axes[1].set_xlabel('Доход')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

display(df_boolean.query("Exited == True").groupby('Age_range')['EstimatedSalary'].agg(['count','mean','median']).reset_index())     # Подсчет ушедших клиентов, среднего значение и медианы их заработной платы
display(df_boolean.query("Exited == False").groupby('Age_range')['EstimatedSalary'].agg(['count','mean','median']).reset_index())    # Подсчет действующих клиентов, среднего значение и медианы их заработной платы

Как видно, различия заработной платы между ушедшими и действующими клиентами практически отсутствуют. Следовательно, заработная плата не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
df['EstimatedSalary'].hist(bins = 25)    # Создание гистограммы клиентов и их заработной платы
plt.xlabel('Доход')
plt.ylabel('Количество клиентов')

На данной гистограмме распределение значений стремится к равномерному. Убедимся в этом наверняка.

In [ ]:
df_salary_uniform_dist = df['EstimatedSalary']

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

sns.kdeplot(df_salary_uniform_dist,ax=axes[0])

mean = df_salary_uniform_dist.mean()
std = df_salary_uniform_dist.std()

axes[0].set_xlabel('Доход')            
axes[0].set_ylabel('Плотность')
axes[0].axvline(mean, color='g', linestyle='--', label='Mean')
axes[0].axvline(mean + std, color='b', linestyle='--', label='Mean + Std')
axes[0].axvline(mean - std, color='b', linestyle='--', label='Mean - Std')
axes[0].legend()

sns.ecdfplot(df_salary_uniform_dist,ax=axes[1])
axes[1].set_xlabel('Доход')            
axes[1].set_ylabel('Плотность')
axes[1].axvline(mean, color='g', linestyle='--', label='Mean')
axes[1].axvline(mean + std, color='b', linestyle='--', label='Mean + Std')
axes[1].axvline(mean - std, color='b', linestyle='--', label='Mean - Std')
axes[1].legend()

plt.show()

In [ ]:
stats.probplot(df_salary_uniform_dist,dist='uniform',plot=plt)
plt.xlabel('Доля')
plt.ylabel('Доход')
plt.title('График совпадений')
plt.show()

В итоге можно утверждать, что заработная плата клиентов имеет равномерное распределение.

In [ ]:
sns.countplot(x='Complain', hue='Exited', data=df_boolean)    # Создание столбчатого графика по наличию жалоб у клиента
plt.xlabel('Наличие жалоб')
plt.ylabel('Количество клиентов')
plt.show()
 
df_complain_true = df_boolean.query("Exited == True").groupby('Age_range')['Complain'].agg(['count','mean']).reset_index()
df_complain_false = df_boolean.query("Exited == False").groupby('Age_range')['Complain'].agg(['count','mean']).reset_index()

df_complain_true = df_complain_true.assign(complain_clients = df_complain_true['count'] * df_complain_true['mean'])
df_complain_false = df_complain_false.assign(complain_clients = df_complain_false['count'] * df_complain_false['mean']) 
display(df_complain_true)
display(df_complain_false) 

Согласно этому набору данных, можно сделать следующие выводы:
* Более 99,6% ушедших клиентов имели жалобы
* Менее 0,01% действующих клиентов имеют жалобы


In [ ]:
sns.countplot(x='Satisfaction Score', hue='Exited', data=df_boolean)
plt.xlabel('Оценка качества обслуживания')
plt.ylabel('Количество клиентов')
plt.show()

df_score = df_boolean.groupby('Satisfaction Score')['Exited'].agg(['count','mean']).reset_index()    # Создание датафрейма с группировкой по оценке качества обслуживания
df_score = df_score.assign(exited_clients = df_score['count'] * df_score['mean'])
display(df_score)

Как видно выше, различия в качестве обслуживания между ушедшими и действующими клиентами практически отсутствуют. Следовательно, оценка качества обслуживания не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(2,1,1)
plt.pie(df.query("Exited == True")['Card Type'].value_counts(), autopct='%1.2f%%', labels = ['Diamond', 'Platinum', 'Silver', 'Gold',], startangle=90)
plt.title('Распределение типов карт для ушедших клиентов')                                                                                  

plt.subplot(2,1,2)
plt.pie(df.query("Exited == False")['Card Type'].value_counts(), autopct='%1.2f%%', labels = ['Diamond', 'Platinum', 'Silver', 'Gold',], startangle=90)
plt.title('Распределение типов карт для действующих клиентов')

plt.show()
                                                           

Как видно выше, распределение карт между ушедшими и действующими клиентами схожи. Следовательно, тип карты не оказывает какого - либо существенного влияния на отток клиентов.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 4))

df_boolean.query("Exited == True")['Point Earned'].hist(ax=axes[0], bins = 25)    # Создание гистограммы ушедших клиентов и количество их бонусов
axes[0].set_xlabel('Накопительные баллы')                                                   
axes[0].set_ylabel('Количество ушедних клиентов')                                         

df_boolean.query("Exited == False")['Point Earned'].hist(ax=axes[1], bins = 25)   # Создание гистограммы действующих клиентов и количество их бонусов
axes[1].set_xlabel('Накопительные баллы')
axes[1].set_ylabel('Количество действующих клиентов')
plt.show()

display(df_boolean.query("Exited == True").groupby('Age_range')['Point Earned'].agg(['count','mean','median']).reset_index())     
display(df_boolean.query("Exited == False").groupby('Age_range')['Point Earned'].agg(['count','mean','median']).reset_index())    

Как видно выше, распределение баллов между ушедшими и действующими клиентами практически совпадает. Следовательно, накопительные баллы не оказывает какого - либо существенного влияния на отток клиентов.

# *Интерпретация результатов исследования и подведение итогов*

В ходе исследование установлено следующее:
* Географическое положение влияет на отток клиентов, причем распределение оттока варьируется в зависимостри от страны. Так, наибольший отток клиентов наблюдается в Германии - 32,44% (814 из 2509 клиентов). В оставшихся странах доля оттока клиентов приблизительно равна - 16,67% для Испании (413 из 2477 клиентов) и 16,17% для Франции (811 из 5014 клиентов).
* Пол клиента также имеет значение. Клиенты женского пола больше склонны к уходу из банка - 25,07% (1139 из 4543 клиентов). У мужчин доля ушедших клиентов равна - 16,47% (899 из 5457 клиентов).
* Для каждой возрастной группы своя доля оттока. Так, молодые клиенты менее склонны к уходу, а с увеличением возраста доля оттока возрастает. Только после 60 лет доля оттока клиентов имеет отрицательный рост. Наибольший отток наблюдается у клиентов в возрасте от 46 до 60 лет  - 51,12% (842 из 1647 клиентов). Предпочтительным клиентом же является мужчина в возрасте от 26 до 35 лет включительно.
* Активность и вовлеченность клиента также влияют на возможный уход. Менее активные клиенты чаще уходят из банка - 26,87% (1303 из 4849 клиентов). Доля ушедших активных клиентов заметно меньше - 14,27% (735 из 5151 клиента).
* Выявлена линейная зависимость ухода клиента с наличием у него жалоб. Так более 99,6% ушедших клиентов имели жалобы. Следует тщательно разобраться в  данной проблеме и попытаться снизить вероятность ухода клиента по этой причине.

Конечно, чтобы получить действительно глубокое представление, мы можем выполнить более детальный анализ, разработку функций и прогнозное моделирование с использованием соответствующих методов и алгоритмов. Но это выходит за рамки моего исследования, которое проводилось сугубо в учебных целях...